In [1]:
import tensorflow as tf

In [1]:
#Implemantar la clase con el modelo base de Logistic Classifier
class LSTM:
    def __init__(self, seq_max_len, state_size, vocab_size, num_class):
        self.seq_max_len = seq_max_len
        self.state_size = state_size
        self.vocab_size = vocab_size
        self.num_classes = num_classes
        
    def build (self):
        self.x = tf.placeholder (shape=[None, self.seq_max_len], dtype=tf.float32)
        x_one_hot_en = tf.one_hot(self.x, self.vocab_size)
        
        #Agregamos la representación temporal del LSTM
        rnn_input = tf.unstack (x_one_hot_en, axis=1)
        
        self.y = tf.placeholder (shape=[None], dtype=tf.float32)
        y_one_hot_en = tf.one_hot(self.y, self.num_classes)
        
        self.batch_size = tf.placeholder(tf.int32, [], name='batch_size')
        
        weights = {
            'layer_0': tf.Variable(tf.random_normal ([self.state_size, 280])),
            'layer_1': tf.Variable(tf.random_normal ([280, self.num_classes ])),
        }
      # y = wx + b
        biases = {
            'layer_0':  tf.Variable(tf.random_normal ([280])),
            'layer_1':  tf.Variable(tf.random_normal ([280, self.num_classes])),
        }
        
        init_state = tf.zeros([self.batch_size, self.state_size]) #debe tener un estado inicial
        #BasicLSTMCell()
        cell = tf.contrib.rnn.BasicLSTMCell(self.state_size)
        self.outputs, self.final_state =  tf.contrib.rnn.static_rnn(cell, rnn_input, dtype=tf.float32)
        
        output = self.outputs[-1]
        
        
        hidden_layer = tf.matmul(output, weights['layer_0']) + biases['layer_0']
        hidden_layer = tf.nn.tanh(hidden_layer)

        self.logits = tf.matmul(hidden_layer, weights['layer_1']) + biases['layer_1']
        #Normalizar los valores que arroja la red
        self.probs = tf.nn.softmax(self.logits, axis=1)
        
        self.correct_preds = tf.equal (tf.argmax(self.probs, axis =1), tf.argmax(self.y_one_hot_en, axis=1))
        self.precission = tf.reduce_mean(correct_preds)
        return 
    
    def step_training (self, learning_rate=0.1):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y_one_hot_en, logits=self.logits))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
        return loss, optimizer
        
    
        
        